In [1]:
import os
if not os.path.exists('enwiki-latest-pages-articles.xml.bz2'):
    download_str = '''#!/bin/sh
set -e

LG=$1
WIKI_DUMP_NAME=${LG}wiki-latest-pages-articles.xml.bz2
WIKI_DUMP_DOWNLOAD_URL=https://dumps.wikimedia.org/${LG}wiki/latest/$WIKI_DUMP_NAME

# download latest Wikipedia dump in chosen language
echo "Downloading the latest $LG-language Wikipedia dump from $WIKI_DUMP_DOWNLOAD_URL..."
wget -c $WIKI_DUMP_DOWNLOAD_URL
echo "Succesfully downloaded the latest $LG-language Wikipedia dump to $WIKI_DUMP_NAME"
    '''
    with open('download_wiki_dump.sh', 'w') as f:
        f.write(download_str)
    !chmod +777 download_wiki_dump.sh
    !./download_wiki_dump.sh en

'.' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
if not os.path.exists('enwiki-latest-pages-articles.txt'):
    processing_str = '''
#!/bin/sh
set -e

WIKI_DUMP_FILE_IN=$1
WIKI_DUMP_FILE_OUT=${WIKI_DUMP_FILE_IN%%.*}.txt

# clone the WikiExtractor repository
git clone https://github.com/attardi/wikiextractor.git

# Master is broken. Check out old version of wikiextractor
git checkout e4abb4cbd019b0257824ee47c23dd163919b731b

# extract and clean the chosen Wikipedia dump
echo "Extracting and cleaning $WIKI_DUMP_FILE_IN to $WIKI_DUMP_FILE_OUT..."
python wikiextractor/WikiExtractor.py $WIKI_DUMP_FILE_IN --processes 8 -q -o - \
| sed "/^\s*\$/d" \
| grep -v "^<doc id=" \
| grep -v "</doc>\$" \
> $WIKI_DUMP_FILE_OUT
echo "Succesfully extracted and cleaned $WIKI_DUMP_FILE_IN to $WIKI_DUMP_FILE_OUT"
'''
    with open('extract_and_clean_wiki_dump.sh', 'w') as f:
        f.write(processing_str)
    !chmod +777 extract_and_clean_wiki_dump.sh
    !./extract_and_clean_wiki_dump.sh enwiki-latest-pages-articles.xml.bz2

'.' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install blingfire


In [4]:
#blingfire text_to_sentence

#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
from pathlib import Path
from blingfire import text_to_sentences

def use_blingfire(text_input):
    wiki_dump_file_in = Path(text_input)
    wiki_dump_file_out = wiki_dump_file_in.parent / \
        f'{wiki_dump_file_in.stem}_preprocessed{wiki_dump_file_in.suffix}'

    print(f'Pre-processing {wiki_dump_file_in} to {wiki_dump_file_out}...')
    with open(wiki_dump_file_out, 'w', encoding='utf-8') as out_f:
        with open(wiki_dump_file_in, 'r', encoding='utf-8') as in_f:
            for line in in_f:
                sentences = text_to_sentences(line)
                out_f.write(sentences + '\n')
    print(f'Successfully pre-processed {wiki_dump_file_in} to {wiki_dump_file_out}...')


if not os.path.exists('enwiki-latest-pages-articles_preprocessed.txt'):
    use_blingfire('enwiki-latest-pages-articles.txt')

Pre-processing enwiki-latest-pages-articles.txt to enwiki-latest-pages-articles_preprocessed.txt...


FileNotFoundError: [Errno 2] No such file or directory: 'enwiki-latest-pages-articles.txt'

In [ ]:
!wc -l enwiki-latest-pages-articles_preprocessed.txt


In [ ]:

for linemag in [4, 5, 6]:
    num_lines = 10**linemag
    print(num_lines)
    !head enwiki-latest-pages-articles_preprocessed.txt -n {num_lines} > sample{num_lines}.txt

In [ ]:
!ls -l
